In [25]:
#разобраться с библиотеками
#считать страницу результатов авито
#считать все объявления этой страницы
#сформировать датафрейм из нужных данных
#считать следующие страницы
#попробовать забирать номера телефонов
#Исключена ошибка повторной попытки считать телефон

In [59]:
#!pip install selenium
#!pip install webdriver-manager
#!pip install pytesseract
#!pip install tesserocr

In [60]:
import pytesseract
from PIL import Image
from time import sleep

In [61]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
import json
from selenium.webdriver.common.action_chains import ActionChains #для скролла до номера телефона

In [62]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Looking for [chromedriver 83.0.4103.39 mac64] driver in cache 
[WDM] - File found in cache by path [/Users/zzema/.wdm/drivers/chromedriver/83.0.4103.39/mac64/chromedriver]


In [63]:
driver.get('https://www.avito.ru/moskva/mebel_i_interer/kompyuternye_stoly_i_kresla-ASgBAgICAURa0E8?cd=1&geoCoords=55.6005021%2C37.7254322&radius=5&user=1')
#webdriver.Chrome(executable_path='/path/to/chromedriver')
#button = driver.find_element_by_xpath('//button[@class="button button-origin button-origin_large button-origin_full-width recommendations-show-more js-recommendations-show-more"]').click()

In [64]:
requiredHtml = driver.page_source

In [65]:
#Получили массив товаров с первой страницы
soup = BeautifulSoup(requiredHtml, 'html5lib')
table=soup.findAll("div", {"class": "item_table-wrapper"})
len(table)

50

In [66]:
table[0] #элемент одного объявления, здесь есть и инфа и и блок с номером телефона, для вычисления айди объявления. которое можно было взять из ссылки, но уже взял из блока

<div class="item_table-wrapper">
  <div class="description item_table-description">
    <div class="snippet-title-row"><h3 class="snippet-title" data-marker="item-title" data-shape="default"><a class="snippet-link" href="/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705" itemprop="url" target="_blank" title="Запчасть от кресла крестовина в Москве">Запчасть от кресла крестовина</a></h3></div>    <div class="snippet-price-row"><span class="snippet-price" data-marker="item-price" data-shape="default">
 500  ₽
  </span></div>              
 <div class="data">
                      <div><div class="item-address" data-shape="default"><div itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span class="item-address__radius" style="color: #5FA800">
 1,5 км
 </span>
   ·       <span class="item-address__string">
 Ореховый б-р, 8
 </span><div class="item-address-georeferences"><span class="item-address-georeferences"><span class="item-address-georeferences-

In [67]:
#Находит айди объявления для того чтобы по нему потом найти кнопку: "показать телефон"
def get_item_id(data):
    try:
        temp=data.find("span", {"class": "js-messenger-button"}).get('data-props')
        return json.loads(temp).get("itemId")
    except:
        temp = data.find('h3').find('a').get('href')
        title = temp[-10:]#Если поля отправить сообщение нет, то айди берем из урла
        return title

In [68]:
get_item_id(table[11])

'1915778253'

In [69]:
# вырезаем из скриншота номер телефона
def crop(location, size,scrolly):
    image = Image.open('screenshot.png')
    x = location['x']
    y = location['y']
    width = size['width']
    height = size['height']
    img=image.crop((x, y-scrolly, x+width, y-scrolly-125+height))#125 Вычитаем, потому что ссылаемся на другой блок Так как блок телефона пропадает из за нежелания оставить номер
    img.save('tel.gif')
    return pytesseract.image_to_string(img)
#для того чтобы телефон попал на скриншот - экран надо проскроллить, соответственно чтобы вырезать из скриншота телефон - надо учесть смещение по y

In [70]:
def phone_extract (driver, table, i):# поиск ячейки с номером, клик по нему, если есть необходимость,
    str1='//button[@data-item-id="'+str(get_item_id(table[i]))+'"]'
    str2="(//div[@class='item_table-aside'])["+str(i+1)+"]"
#Проверяем, не открывали ли мы телефон ранее
    try:
        temp1=driver.find_element_by_xpath(str1)
        temp1.click()
        print('vse norm')
    except:
#        temp1=driver.find_element_by_xpath(str2)#id+1 так как массив с единицы
        print("doesn't need to click")
#Скроллим до нужного места в выдаче, удобнее чем заходить в каждое объявление и брать номер оттуда
    sleep(0.5)
    element = driver.find_element_by_xpath(str2)
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
#Скриншотим экран, чтобы взять номер
    driver.save_screenshot('screenshot.png')
#для того чтобы телефон попал на скриншот - экран надо проскроллить, соответственно чтобы вырезать из скриншота телефон - надо учесть смещение по y
    scrolly=driver.execute_script("return window.scrollY;")
#определяем местоположение и рамзер номера телефона
    #temp2=driver.find_elements_by_xpath('//img[@class="item-extended-phone"]')
    location = element.location #метоположение телефона на странице
    size = element.size #размер картинки с номером телефона
    return crop(location, size,scrolly)

In [71]:
data1=[]
for i, ad in enumerate(table):
        try:
            div = ad.find('h3')
            title = div.text.strip()
        except:
            title = ''
        try:
            div = ad.find('h3')
            url = "https://avito.ru" + div.find('a').get('href')
        except:
            url = ''
        try:
            price = ad.find("span", {"class": "snippet-price"}).text.strip('  ₽\r\n')
        except:
            price = ''
        try:
            div = ad.find('span', class_='item-address-georeferences-item__content')
            metro = div.text.strip()
        except:
            metro = ''
        try:
            item_id = get_item_id(ad)
        except:
            item_id = ''
        data = {'title':title,
                'price':price,
                'metro':metro,
                'url':url,
                'item_id':item_id,
               'phone':phone_extract (driver, table, i)}
        data1.append(data)
        print(data1)
        sleep(3)

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}]
vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}]
vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https:/

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

...

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

vse norm
[{'title': 'Запчасть от кресла крестовина', 'price': '500', 'metro': 'Домодедовская', 'url': 'https://avito.ru/moskva/mebel_i_interer/zapchast_ot_kresla_krestovina_1061845705', 'item_id': 1061845705, 'phone': '8.901 779-88-19'}, {'title': 'Кресло на колесиках', 'price': '400', 'metro': 'Орехово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_na_kolesikah_1937993379', 'item_id': 1937993379, 'phone': '8 929 637-80-79'}, {'title': 'Продам письменный стол', 'price': '3 000', 'metro': 'Алма-Атинская', 'url': 'https://avito.ru/moskva/mebel_i_interer/prodam_pismennyy_stol_1512085242', 'item_id': 1512085242, 'phone': '8 963 623-26-26'}, {'title': 'Кресло', 'price': '1 600', 'metro': 'Зябликово', 'url': 'https://avito.ru/moskva/mebel_i_interer/kreslo_1911622305', 'item_id': 1911622305, 'phone': ''}, {'title': 'Компьютерный стол Хофф Hoff', 'price': '1 999', 'metro': 'Шипиловская', 'url': 'https://avito.ru/moskva/mebel_i_interer/kompyuternyy_stol_hoff_hoff_1932578252', 'item_id

In [73]:
import pandas as pd
data_pd=pd.DataFrame(data1)
data_pd.head(50)

,title,price,metro,url,item_id,phone
0,Запчасть от кресла крестовина,500,Домодедовская,https://avito.ru/moskva/mebel_i_interer/zapcha...,1061845705,8.901 779-88-19
1,Кресло на колесиках,400,Орехово,https://avito.ru/moskva/mebel_i_interer/kreslo...,1937993379,8 929 637-80-79
2,Продам письменный стол,3 000,Алма-Атинская,https://avito.ru/moskva/mebel_i_interer/prodam...,1512085242,8 963 623-26-26
3,Кресло,1 600,Зябликово,https://avito.ru/moskva/mebel_i_interer/kreslo...,1911622305,
4,Компьютерный стол Хофф Hoff,1 999,Шипиловская,https://avito.ru/moskva/mebel_i_interer/kompyu...,1932578252,
5,Компьютерный стул,500,Красногвардейская,https://avito.ru/moskva/mebel_i_interer/kompyu...,1937307220,8 926 759-04-52
6,Компьютерный стол,1 500,Красногвардейская,https://avito.ru/moskva/mebel_i_interer/kompyu...,1937563709,8 926 759-04-52
7,Компьютерный стол школьный стол,4 500,Зябликово,https://avito.ru/moskva/mebel_i_interer/kompyu...,1939055124,8 926 172-66-75
8,Стул-кресло IKEA фанбюн,4 000,Орехово,https://avito.ru/moskva/mebel_i_interer/stul-k...,1598997901,8919 479-93-22
9,Стол обеденный или письменный IKEA,6 000,Орехово,https://avito.ru/moskva/mebel_i_interer/stol_o...,1643534531,8919 479-93-22


In [126]:
phone_extract (driver, table, 5)

doesn't need to click


'8 926 759-04-52'

In [70]:
driver.execute_script("return window.scrollY;")

76